######The latest version of this IPython notebook is available at [http://github.com/jckantor/CBE20255](http://github.com/jckantor/CBE20255)  for noncommercial use under terms of the [Creative Commons Attribution Noncommericial ShareAlike License](http://creativecommons.org/licenses/by-nc-sa/4.0/).

J.C. Kantor (Kantor.1@nd.edu)

# Adipic Acid Flowsheet

This [IPython notebook](http://ipython.org/notebook.html) demonstrates the formulation and solution of material balances for a hypothetical adipic acid process described by Murphy (2005, Example 2.15) using the [symbolic algebra package Sympy](http://sympy.org/en/index.html).

## Problem Statement

[Adipic acid](http://pubchem.ncbi.nlm.nih.gov/summary/summary.cgi?cid=196) (C<sub>6</sub>H<sub>10</sub>O<sub>4</sub>) (also called hexanedioic acid) rarely occurs in nature, but approximately 2.5 billion kilograms are produced per year from petrochemical feedstocks primarily for the production of nylon.

<img src="http://upload.wikimedia.org/wikipedia/commons/thumb/7/7e/Adipic-acid-3D-balls.png/320px-Adipic-acid-3D-balls.png" width = 200px/>

Recently there has been interest in [producing adipic acid from renewable resources](http://www.ihs.com/products/chemical/technology/pep/bio-based-adipic-acid.aspx). [For example, starting with glucose](http://pubs.acs.org/doi/abs/10.1021/ja00080a057) (C<sub>6</sub>H<sub>12</sub>O<sub>6</sub>), muconic acid (C<sub>6</sub>H<sub>6</sub>O<sub>4</sub>) is produced through fermentation with a genetically modified e. coli. via the reaction

<sup>7</sup>&frasl;<sub>3</sub> C<sub>6</sub>H<sub>12</sub>O<sub>6</sub> + <sup>17</sup>&frasl;<sub>2</sub> O<sub>2</sub> &#10141; C<sub>6</sub>H<sub>6</sub>O<sub>4</sub> + 8 CO<sub>2</sub> + 11 H<sub>2</sub>O

that is subsequently hydrogentated to form adipic acid

C<sub>6</sub>H<sub>6</sub>O<sub>4</sub> + 2 H<sub>2</sub>  &#10141; C<sub>6</sub>H<sub>10</sub>O<sub>4</sub>

Murphy (2005, Example 2.15) outlines a hypothetical flowsheet for this process:

<img src = "https://raw.github.com/jckantor/CBE20255/0d26361216f76b42f875bc066825d6c79b412583/images/adipic_acid_flowsheet.png" width=800px>

Neglecting the _E. coli_, solve for the flowrates necessary to produce 12,000 kg/hour of adipic acid assuming that glucose is available as a 10 mg/mL solution.


## Solution

### Process Variables

We begin by relabeling the process flowsheet with stream numbers, stream variables, and extents of reaction. There are chemical species are abbreviated as follows:

* A: adipic acid
* C: carbon dioxide
* G: glucose
* H: hydrogen
* M: muconic acid
* N: nitrogen
* O: oxygen
* W: water

and where X<sub>1</sub> and X<sub>2</sub> denote the extents of reactions 1 and 2, respectively. The stream variables denote chemical flowrates in units of kg/hour. The extents of reaction will be in units of kg-mol/hour.

<img src = "https://raw.github.com/jckantor/CBE20255/0d26361216f76b42f875bc066825d6c79b412583/images/adipic_acid_flowsheet_2.png" width=800px>

In [10]:
# Import the symbolic algebra package sympy
import sympy as sym

# Extents of reactions 1 and 2
sym.var('X1 X2')

# Stream variables
sym.var('O1 N1')           # Stream 1
sym.var('G2 W2')           # Stream 2
sym.var('H3')              # Stream 3
sym.var('N4 C4')           # Stream 4
sym.var('W5')              # Stream 5
sym.var('A6')              # Stream 6
sym.var('O7 N7 G7 W7')     # Stream 7
sym.var('N8 W8 C8 M8')     # Stream 8
sym.var('M9')              # Stream 9
sym.var('H10 M10')         # Stream 10

(H10, M10)

Because the flowsheet includes reactions, it will be necessary to include molecular weights in the mass balance expressions. For this purpose we gather the molecular weights of all species into a python dictionary indexed by the chemical species.

In [11]:
MW = {
    'A': 146.14,
    'C': 44.01,
    'G': 180.16,
    'H': 2.02,
    'M': 142.11,
    'N': 14.01,
    'O': 16.00,
    'W': 18.02
}

### Specifications

In [12]:
specs = [
    sym.Eq(A6, 12000),
    sym.Eq(N1/MW['N'], (0.79/0.21)*(O1/MW['O'])),
    sym.Eq(G2, 0.01*W2)
]

### Material Balances

In [13]:
mixer1 = [
    sym.Eq(0, O1 - O7),
    sym.Eq(0, N1 - N7),
    sym.Eq(0, G2 - G7),
    sym.Eq(0, W2 - W7)
]

reactor1 = [
    sym.Eq(0,  O7 - MW['O']*(17/2)*X1),
    sym.Eq(0,  N7 - N8),
    sym.Eq(0,  G7 - MW['G']*(7/3)*X1),
    sym.Eq(0, -C8 + MW['C']*8*X1),
    sym.Eq(0, -M8 + MW['M']*X1),
    sym.Eq(0,  W7 - W8 + MW['W']*11*X1)
]

separator = [
    sym.Eq(0, N8 - N4),
    sym.Eq(0, C8 - C4),
    sym.Eq(0, M8 - M9),
    sym.Eq(0, W8 - W5)
]

mixer2 = [
    sym.Eq(0, M9 - M10),
    sym.Eq(0, H3 - H10)
]

reactor2 = [
    sym.Eq(0, H10 - MW['H']*2*X2),
    sym.Eq(0, M10 - MW['M']*X2),
    sym.Eq(0, -A6 + MW['A']*X2)
]

mbals = mixer1 + reactor1 + separator + mixer2 + reactor2

In [14]:
specs + mbals

[A6 == 12000,
 0.0713775874375446*N1 == 0.235119047619048*O1,
 G2 == 0.01*W2,
 0 == O1 - O7,
 0 == N1 - N7,
 0 == G2 - G7,
 0 == W2 - W7,
 0 == O7 - 128.0*X1,
 0 == N7 - N8,
 0 == G7 - 360.32*X1,
 0 == -C8 + 352.08*X1,
 0 == -M8 + 142.11*X1,
 0 == W7 - W8 + 198.22*X1,
 0 == -N4 + N8,
 0 == -C4 + C8,
 0 == M8 - M9,
 0 == -W5 + W8,
 0 == -M10 + M9,
 0 == -H10 + H3,
 0 == H10 - 4.04*X2,
 0 == M10 - 142.11*X2,
 0 == -A6 + 146.14*X2]

### Solution

In [15]:
soln = sym.solve(mbals + specs)
soln

{N8: 34621.6739330193,
 O7: 10510.4694128917,
 G2: 29586.9713972903,
 O1: 10510.4694128917,
 X1: 82.1130422882168,
 W5: 2974973.58697140,
 A6: 12000.0000000000,
 N4: 34621.6739330193,
 H3: 331.736690844396,
 G7: 29586.9713972903,
 W8: 2974973.58697140,
 W2: 2958697.13972903,
 N1: 34621.6739330193,
 C8: 28910.3599288354,
 N7: 34621.6739330193,
 M10: 11669.0844395785,
 H10: 331.736690844396,
 W7: 2958697.13972903,
 M8: 11669.0844395785,
 X2: 82.1130422882168,
 M9: 11669.0844395785,
 C4: 28910.3599288354}

In [16]:
for key in soln.keys():
    print "{:3s}    {:10.1f}".format(key, float(soln[key]))

N8        34621.7
O7        10510.5
G2        29587.0
O1        10510.5
X1           82.1
W5      2974973.6
A6        12000.0
N4        34621.7
H3          331.7
G7        29587.0
W8      2974973.6
W2      2958697.1
N1        34621.7
C8        28910.4
N7        34621.7
M10       11669.1
H10         331.7
W7      2958697.1
M8        11669.1
X2           82.1
M9        11669.1
C4        28910.4
